In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_vertical_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,36451.08,0.69,0.19,0.12,0.01,0.00,-0.00,-0.00
3,0.69,8126.96,-0.08,0.05,-0.00,0.00,-0.00,0.00
4,0.19,-0.08,1515.04,0.01,0.00,-0.00,0.00,-0.00
5,0.12,0.05,0.01,252.52,0.00,0.00,0.00,-0.00
6,0.01,-0.00,0.00,0.00,36.30,0.00,0.00,0.00
7,0.00,0.00,-0.00,0.00,0.00,4.81,-0.00,0.00
8,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.62,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.07


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00003,0.00004,0.00000,0.00000,-0.00000,-0.00001
3,0.00004,1.00000,-0.00002,0.00003,-0.00000,0.00001,-0.00003,0.00003
4,0.00003,-0.00002,1.00000,0.00002,0.00001,-0.00005,0.00004,-0.00002
5,0.00004,0.00003,0.00002,1.00000,0.00002,0.00002,0.00001,-0.00002
6,0.00000,-0.00000,0.00001,0.00002,1.00000,0.00002,0.00002,0.00002
7,0.00000,0.00001,-0.00005,0.00002,0.00002,1.00000,-0.00003,0.00001
8,-0.00000,-0.00003,0.00004,0.00001,0.00002,-0.00003,1.00000,0.00002
9,-0.00001,0.00003,-0.00002,-0.00002,0.00002,0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.0335823797500379

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.66266611e+08 8.08731459e+06 2.73288022e+05 7.28858714e+03
 1.41300672e+02 2.21029074e+00 2.78606824e-02 2.61417407e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999423,-0.033426,-0.005888,-0.000977,-0.000141,-0.000019,-0.000002,-2.794939e-07
1,0.033213,0.998910,-0.032366,-0.005174,-0.000744,-0.000098,-0.000013,-1.317270e-06
2,0.006919,0.031952,0.998861,-0.034410,-0.004785,-0.000637,-0.000081,-9.289426e-06
3,0.001379,0.006201,0.034017,0.998711,-0.036808,-0.004754,-0.000614,-6.874845e-05
4,0.000248,0.001113,0.005969,0.036364,0.998289,-0.045034,-0.005685,-6.326868e-04
5,0.000044,0.000195,0.001053,0.006301,0.044353,0.996811,-0.065636,-7.129060e-03
6,0.000008,0.000038,0.000201,0.001210,0.008424,0.064118,0.991668,-1.114049e-01
7,0.000002,0.000008,0.000046,0.000273,0.001896,0.014310,0.110697,9.937493e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005766533451398992,
 0.0010896001571589764,
 0.0011387270089984192,
 0.0012885028198729165,
 0.0017114849949092514,
 0.0031885528778015093,
 0.008332023800179167,
 0.006250677481770661]